## <font color=blue> ::Load Modules:: </font>


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from netCDF4 import Dataset
from pandas.plotting import table 
import xarray as xr
from datetime import date
import time
import os

<div style="border: 2px solid black; border-bottom: none; padding: 20px;">

<span style="font-family: 'Times New Roman'; font-size: 32px; color: #0000FF;">
Emulandice AIS and GIS .nc data</span>

---

<span style="font-family: 'Arial';">
    
</span>


</div>

<span style="font-family: 'Times New Roman'; font-size: 24px; color: #FF0000;">
    _> Import data
</span>

In [2]:
# Choose ssp
scenario = 585
#
# Import the NC data.
d_ncAIS = xr.open_dataset('/projects/kopp/facts-experiments/221217/coupling.ssp{arg1}/output/coupling.ssp{arg1}.emuAIS.emulandice.AIS_globalsl.nc'.format(arg1=scenario))
d_ncEAIS = xr.open_dataset('/projects/kopp/facts-experiments/221217/coupling.ssp{arg1}/output/coupling.ssp{arg1}.emuAIS.emulandice.AIS_EAIS_globalsl.nc'.format(arg1=scenario))
d_ncWAIS = xr.open_dataset('/projects/kopp/facts-experiments/221217/coupling.ssp{arg1}/output/coupling.ssp{arg1}.emuAIS.emulandice.AIS_WAIS_globalsl.nc'.format(arg1=scenario))
d_ncPEN = xr.open_dataset('/projects/kopp/facts-experiments/221217/coupling.ssp{arg1}/output/coupling.ssp{arg1}.emuAIS.emulandice.AIS_PEN_globalsl.nc'.format(arg1=scenario))
d_ncGIS = xr.open_dataset('/projects/kopp/facts-experiments/221217/coupling.ssp{arg1}/output/coupling.ssp{arg1}.emuGrIS.emulandice.GrIS_globalsl.nc'.format(arg1=scenario))
d_ncAIS
#
# Labels.
d_ncAIS_nme  = d_ncAIS.attrs['source'].split(':')[1].split('coupling.')[1]
d_ncGIS_nme = d_ncGIS.attrs['source'].split(':')[1].split('coupling.')[1]
Filesource= d_ncAIS_nme+" , "+d_ncGIS_nme


<span style="font-family: 'Times New Roman'; font-size: 24px; color: #FF0000;">
    _> Compute percentile
</span>

In [3]:
def ptile(slc):
    slc_P50=np.percentile(slc, 50, axis=0)
    slc_P17=np.percentile(slc, 17, axis=0)
    slc_P83=np.percentile(slc, 83, axis=0)
    slcOUT=np.vstack((slc_P50,slc_P17,slc_P83))
    return slcOUT
#
slcAIS_ptile=ptile(d_ncAIS.sea_level_change[:,:,0].values)
slcEAIS_ptile=ptile(d_ncEAIS.sea_level_change[:,:,0].values)
slcWAIS_ptile=ptile(d_ncWAIS.sea_level_change[:,:,0].values)
slcPEN_ptile=ptile(d_ncPEN.sea_level_change[:,:,0].values)
slcGIS_ptile=ptile(d_ncGIS.sea_level_change[:,:,0].values)

<span style="font-family: 'Times New Roman'; font-size: 24px; color: #FF0000;">
    _> Save Other variables
</span>

In [4]:
years=d_ncAIS.years.values
samples=d_ncAIS.samples.values
locations=d_ncAIS.locations.values
#
quantiles=np.array([50,17,83])

<span style="font-family: 'Times New Roman'; font-size: 24px; color: #FF0000;">
    _> OutPut NC file of quantiles
</span>

In [5]:
# NC file Name
nc_filename = os.getcwd()+"/emulandiceAIS_GIS_ssp{arg1}_globalsl.nc".format(arg1=scenario)
rootgrp = Dataset(nc_filename, "w", format="NETCDF4")
#
# Define Dimensions
year_dim = rootgrp.createDimension("years", len(years))
quant_dim = rootgrp.createDimension("quantiles", len(quantiles))
#
# Populate dimension variables
year_var = rootgrp.createVariable("years", "i4", ("years",))
quant_var = rootgrp.createVariable("quantiles", "i8", ("quantiles",))

# Create a data variable
sampsAIS = rootgrp.createVariable("AIS_sea_level_change", "i2", ("quantiles", "years"), zlib=True, complevel=4)
sampsEAIS = rootgrp.createVariable("EAIS_sea_level_change", "i2", ("quantiles", "years"), zlib=True, complevel=4)
sampsWAIS = rootgrp.createVariable("WAIS_sea_level_change", "i2", ("quantiles", "years"), zlib=True, complevel=4)
sampsPEN = rootgrp.createVariable("PEN_sea_level_change", "i2", ("quantiles", "years"), zlib=True, complevel=4)
sampsGIS = rootgrp.createVariable("GIS_sea_level_change", "i2", ("quantiles", "years"), zlib=True, complevel=4)

# Assign attributes
rootgrp.description = "Global SLR contribution from Antarctica using the emulandice module"
rootgrp.history = "Created " + time.ctime(time.time())
rootgrp.source = Filesource
rootgrp.scenario = "ssp"+str(scenario)
rootgrp.baseyear = 2005
rootgrp.units = "mm"

# Put the data into the netcdf variables
year_var[:] = years
quant_var[:] = quantiles
#
sampsAIS[:,:]  =  slcAIS_ptile[:,:]
sampsEAIS[:,:] = slcEAIS_ptile[:,:]
sampsWAIS[:,:] = slcWAIS_ptile[:,:]
sampsPEN[:,:]  =  slcPEN_ptile[:,:]
sampsGIS[:,:]  =  slcGIS_ptile[:,:]
#
rootgrp.close()

In [6]:
d_nc1=xr.open_dataset(os.getcwd()+"/emulandiceAIS_GIS_ssp{arg1}_globalsl.nc".format(arg1=scenario))
d_nc1

<xarray.Dataset>
Dimensions:                (years: 9, quantiles: 3)
Coordinates:
  * years                  (years) int32 2020 2030 2040 2050 ... 2080 2090 2100
  * quantiles              (quantiles) int64 50 17 83
Data variables:
    AIS_sea_level_change   (quantiles, years) int16 ...
    EAIS_sea_level_change  (quantiles, years) int16 ...
    WAIS_sea_level_change  (quantiles, years) int16 ...
    PEN_sea_level_change   (quantiles, years) int16 ...
    GIS_sea_level_change   (quantiles, years) int16 ...
Attributes:
    description:  Global SLR contribution from Antarctica using the emulandic...
    history:      Created Tue Jun 13 15:01:17 2023
    source:       ssp585.emuAIS.emulandice.AIS.  , ssp585.emuGrIS.emulandice....
    scenario:     ssp585
    baseyear:     2005
    units:        mm